In [1]:
import os
import time
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException

from urllib.parse import urlparse
from urllib.error import URLError
import urllib.robotparser as urobot

from urllib.error import URLError
from datetime import datetime
from selenium.common.exceptions import WebDriverException

import requests
from requests.packages import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from entities import Site, Page, PageData, Link
import repository
import helper_functions

## Set Up Parameters

In [2]:

# Selenium set up
path = os.path.dirname(os.path.abspath("__file__"))
WEB_DRIVER_LOCATION = os.path.join(path, "chromedriver") # chromedriver.exe should be placed inside crawler folder
USER_AGENT = "fri-wier-jernejtim"

chrome_options = Options()
chrome_options.add_argument(f"user-agent={USER_AGENT}")
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')
# chrome_options.add_argument("--headless")

# Crawler set up
# FRONTIER_FILENAME = "frontier.txt"
# CRAWLED_FILENAME = "crawled.txt"

LIMIT_DOMAIN = ".gov.si"
SEED_URLS = ["http://www.gov.si/", "http://www.evem.gov.si/", "http://e-uprava.gov.si/", "http://e-prostor.gov.si/"]
BINARY_CONTENT = ['pdf', 'doc', 'docx', 'ppt', 'pptx']
BANNED_FILETYPES = ['zip', 'jsp']
ALLOWED_LINK_TYPES = [
    'text/html', 
    'application/pdf',
    'application/msword',
    'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
    'application/vnd.ms-powerpoint',
    'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet']

TIMEOUT = 0 # Default timeout if no robots.txt

In [3]:
# if not os.path.isfile(FRONTIER_FILENAME):
#     write_file(FRONTIER_FILENAME, SEED_URLS)

# if not os.path.isfile(CRAWLED_FILENAME):
#     write_file(CRAWLED_FILENAME, '')

In [4]:
def get_type_code(url, html_content):
    duplicate_url = repository.check_if_duplicate(html_content)
    if url.split('.')[-1] in BINARY_CONTENT:
        return 'BINARY', None
    elif url in SEED_URLS:
        return 'FRONTIER', None
    elif duplicate_url:
        return 'DUPLICATE', duplicate_url
    else:
        return 'HTML', None

In [5]:
repository.clear_db()

In [6]:
frontier = [(None, url) for url in SEED_URLS.copy()]

curr_domain = None # Current domain (for updating robots.txt)
errors = []

rp = urobot.RobotFileParser() # robot file parser

driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
# driver.implicitly_wait(TIMEOUT)

while frontier:
    # Get a pair of source url and destination url from frontier
    src_url, dest_url = frontier.pop(0)
    o = urlparse(dest_url)

    # If the domain is different than from the previous url
    # -> Read and parse robots.txt file again
    if o.hostname != curr_domain:
        print(f"Reading 'robots.txt' for domain '{o.hostname}'")
        rp, robots_content, sitemap_content = helper_functions.parse_robots_file(dest_url)
        crawl_delay = rp.crawl_delay(USER_AGENT)
        if crawl_delay: TIMEOUT = crawl_delay
        curr_domain = o.hostname

        # Add new site if on new domain
        if not repository.check_if_site_exists(curr_domain):
            new_site = Site(o.hostname, robots_content, sitemap_content)
            repository.create_site(new_site)

    # If this page was already crawled
    # -> skip this page
    if repository.check_if_page_exists(dest_url):
        print(f"Already Crawled URL '{dest_url}'")
        repository.create_link(Link(src_url, dest_url))
        continue

    # If robots.txt doesnt allow for crawling the page
    # -> skip this page
    if not rp.can_fetch(USER_AGENT, dest_url):
        print(f"Prohibited access to URL '{dest_url}'")
        continue
    
    # Try and fetch the page with the url
    # If it fails -> skip it
    try:
        print(f"Retrieving web page URL '{dest_url}'")
        driver.get(dest_url)
        time.sleep(TIMEOUT)
    except WebDriverException as e:
        print(e)
        errors.append(e)
        continue


    type_code, duplicate_url = get_type_code(dest_url, driver.page_source)

    if type_code == 'FRONTIER':
        site_domain = curr_domain
        html_content = driver.page_source
        status_code = requests.get(dest_url, verify=False).status_code
        accessed_time = datetime.now()
        images = helper_functions.get_all_images(driver, dest_url)
        links = helper_functions.get_all_links(driver, LIMIT_DOMAIN, ALLOWED_LINK_TYPES)
        page_data = None
    
    elif type_code == 'HTML':
        site_domain = curr_domain
        html_content = driver.page_source
        status_code = requests.get(dest_url, verify=False).status_code
        accessed_time = datetime.now()
        images = helper_functions.get_all_images(driver, dest_url)
        links = helper_functions.get_all_links(driver, LIMIT_DOMAIN, ALLOWED_LINK_TYPES)
        page_data = None

    elif type_code == 'BINARY':
        site_domain = curr_domain
        html_content = None
        status_code = requests.get(dest_url, verify=False).status_code
        accessed_time = datetime.now()
        images = []
        links = []        
        page_data = helper_functions.get_page_data(dest_url)

    elif type_code == 'DUPLICATE':
        site_domain = curr_domain
        html_content = None
        status_code = requests.get(dest_url, verify=False).status_code
        accessed_time = datetime.now()
        images = []
        links = []        
        page_data = None
        src_url = duplicate_url


    # Create the page with gathered data 
    # and add it to pages
    page = Page(curr_domain, type_code, dest_url, html_content, status_code, accessed_time, images, page_data)

    repository.create_page(page)
    if images: repository.create_images(images)
    if page_data: repository.create_page_data(page_data)

    # Add the link between the two pages
    repository.create_link(Link(src_url, dest_url))

    print(page)
    print(links)
    print('\n')

    # Add gathered links to the frontier
    d_links = [(dest_url, link) for link in links]
    frontier.extend(d_links)

driver.close()


Reading 'robots.txt' for domain 'www.gov.si'
Retrieving web page URL 'http://www.gov.si/'
site: www.gov.si 
type_code: FRONTIER 
url: http://www.gov.si/ 
html_content: 29545 
status_code: 200 
accessed_time: 2022-03-24 15:30:43.837323 
images: 10 
page_data: None
['https://www.gov.si/#content', 'https://www.gov.si/', 'https://www.gov.si/podrocja/', 'https://www.gov.si/drzavni-organi/', 'https://www.gov.si/zbirke/', 'https://www.gov.si/dogodki/', 'https://www.gov.si/novice/', 'https://www.gov.si/sodelujte/', 'https://www.gov.si/dostopnost/', 'https://www.gov.si/o-spletnem-mestu/', 'https://www.gov.si/drzavni-organi/vlada/', 'https://www.gov.si/zbirke/delovna-mesta/', 'https://www.gov.si/drzave/', 'https://www.gov.si/teme/drzavni-prazniki-in-dela-prosti-dnevi/', 'https://www.gov.si/zbirke/javne-objave/', 'https://www.gov.si/zbirke/projekti-in-programi/pregled-vladnih-projektov-za-razvoj-obcin/', 'https://www.gov.si/teme/pomoc-slovenije-drzavljanom-ukrajine/', 'https://www.gov.si/podrocja

KeyboardInterrupt: 

In [ ]:
# driver = webdriver.Chrome(WEB_DRIVER_LOCATION, options=chrome_options)
# driver.get('https://spot.gov.si/')

# images = helper_functions.get_all_images(driver, 'https://spot.gov.si/')

# for i in images:
#     print(i)